In [6]:
import re

import cohere
import numpy as np
import pandas as pd
import altair as alt
from annoy import AnnoyIndex
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz

ModuleNotFoundError: No module named 'cohere'

In [8]:
df=pd.read_csv("data-set/cities_r2.csv")
df=df["ascii_name"]
df=df.drop_duplicates()
df=df.dropna()
df=df.to_frame(name="ascii_name")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 492 entries, 0 to 492
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ascii_name  492 non-null    object
dtypes: object(1)
memory usage: 7.7+ KB


,ascii_name
0,Abohar
1,Achalpur
2,Adilabad
3,Adityapur
4,Adoni


In [9]:
embeds = co_client.embed(
    texts=list(df['ascii_name']),
    model='large',
    truncate='RIGHT'
).embeddings

embeds = np.array(embeds)
Debug: print(embeds.shape) 

NameError: name 'co_client' is not defined

In [ ]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10)
search_index.save('test.ann')

In [ ]:
EXAMPLE_ID = 69  # Nice
NUM_NEIGHBORS = 30
QUERY = "Gajiabad"   #  Write your own query here(Eg: What is the oldest profession?).

query_embed = co_client.embed(
    texts = [QUERY],
    model = 'large',
    truncate='RIGHT'
).embeddings

similar_item_ids = search_index.get_nns_by_vector(
    query_embed[0], NUM_NEIGHBORS, include_distances=True)

results = pd.DataFrame(
    data={
        'texts': df.iloc[similar_item_ids[0]]['ascii_name'],
        'distance': similar_item_ids[1],
    }
)

print(f"The Question is : {QUERY}")
print('The nearest neighbors are : ')
results

In [ ]:
final=[]
for word in results["texts"]:
    final.append([(fuzz.ratio(QUERY,word)+fuzz.partial_ratio(QUERY,word))/2,word])
final.sort(reverse=True)
final


In [ ]:
from pyphonetics import RefinedSoundex
rs = RefinedSoundex()
final=[]
for word in results["texts"]:
    final.append([rs.distance(QUERY,word),word])
final.sort()
final